In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [52]:
newzy = pd.read_csv('../FinalProject/newzy.csv', sep='|', index_col='doc_id')

/Users/matthewthomas/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
articles = pd.read_csv('../FinalProject/articles.csv', index_col=0)

In [4]:
articles = articles['0'].to_list()

In [19]:
newzy.doc_source.unique()

array(['Google News', 'PowerLine', 'Reuters', 'Guardian', 'Daily Kos',
       'Drudge Report', 'Real Clear Politics', 'NPR', 'Fox',
       'New York Times', 'Breitbart', 'CNN', 'US News', 'UPI Latest',
       'Politico Magazine', nan], dtype=object)

In [53]:
filters =  ['Powerline', 'Daily Kos',\
            'Breitbart', 'Politico Magazine']

In [54]:
newzy = newzy[newzy.doc_source.isin(filters)]

In [55]:
newzy

,doc_source,doc_title,doc_content,doc_date,doc_url
doc_id,,,,,
96,Daily Kos,Obama has done nothing to address income inequ...,Income and wealth inequality is a festering wo...,11/03/2013,http://feeds.dailykos.com/~r/dailykos/index/~3...
97,Daily Kos,"Anti-racism is not 'anti-white,' no matter wha...",The virulent meme spread across our nation wit...,11/03/2013,http://feeds.dailykos.com/~r/dailykos/index/~3...
98,Daily Kos,Abbreviated Pundit Round-up: Angry does no...,Chrystia Freeland sees a fight that's not not ...,11/03/2013,http://feeds.dailykos.com/~r/dailykos/index/~3...
99,Daily Kos,Sunday Talk: Haters gonna hate,"No doubt, the bestest thing about America is o...",11/03/2013,http://feeds.dailykos.com/~r/dailykos/index/~3...
100,Daily Kos,Gore on fire: Says humans treating atmosphere ...,About time Al Gore has been everywhere lately....,11/03/2013,http://feeds.dailykos.com/~r/dailykos/index/~3...
...,...,...,...,...,...
1026288,Breitbart,"'Moderates' Pete Buttigieg, Amy Klobuchar Skip...","Former South Bend, Indiana, Mayor Pete Buttigi...",02/27/2020,http://feedproxy.google.com/~r/breitbart/~3/CF...
1026289,Breitbart,HHS Secretary Defends President Trump at Coron...,HHS Secretary Alex Azar corrected the record a...,02/27/2020,http://feedproxy.google.com/~r/breitbart/~3/OD...
1026290,Breitbart,CDC Confirms Potential Case of Community Sprea...,The Centers for Disease Control and Prevention...,02/27/2020,http://feedproxy.google.com/~r/breitbart/~3/FQ...


In [65]:
breitbart = newzy[newzy.doc_source == 'Breitbart'].reset_index()
breitbart.iloc[5].doc_content

'I\'m working my way through Charles Krauthammer\'s Things That Matter, a collection of his columns over the past three decades. The book\'s publication has provided the occasion for some much-deserved appreciation of the man, whose personal story is remarkable and whose influence on politics is unique. (In what now seems like ancient history, he suggested a plan for averting a debt ceiling crisis in early 2013, which worked.) Still, it feels as though few reviewers have identified what, exactly, makes Krauthammer special. It\'s not just his remarkable recovery from a disabling accident, which Chris Wallace discussed on Fox News Sunday. He does not define the conservative agenda the way Rush Limbaugh or Mark Levin sometimes do. While he\'s no David Brooks, telling the media what they want to hear, he sometimes dissents from the conservative line. What Krauthammer does, above all, is translate\xa0conservatism into the conventional language of mainstream political debate. Conservatives t

In [69]:
len(breitbart[breitbart.doc_content.str.len() > 228])

3041

In [46]:
articles = []
def parse_text(links):
    for link in links:
        url = link
        try:
            r = requests.get(url) #request documentation for errors
        except requests.exceptions.RequestException:
            articles.append('bad link')
        else:
            html_doc = r.text
            soup = BeautifulSoup(html_doc) #check error here
            p_tags = soup.find_all('p')
            p_tags_text = [tag.get_text().strip() for tag in p_tags]
            #Some links don't work so we need to check for that
            if len(p_tags_text) == 0:
                articles.append('bad link')
            else:
                # Filter out sentences that contain newline characters '\n' or don't contain periods.
                sentence_list = [sentence for sentence in p_tags_text if not '\n' in sentence]
                sentence_list = [sentence for sentence in sentence_list if '.' in sentence]
                # Combine list items into string.
                article = ' '.join(sentence_list)
                articles.append(article)

In [23]:
def url_to_string(links):
    for url in links:
        res = requests.get(url)
        html = res.text
        soup = BeautifulSoup(html, 'html5lib')
        for script in soup(["script", "style", 'aside']):
            script.extract()
        article = " ".join(re.split(r'[\n\t]+', soup.get_text()))
        articles.append(article)

In [57]:
links = breitbart.iloc[:101].doc_url 

In [58]:
parse_text(links)

In [59]:
len(articles)

101

In [39]:
articles = pd.Series(articles)

In [40]:
articles.to_csv('articles.csv')

In [62]:
articles[75:]

['This process is automatic. Your browser will redirect to your requested content shortly.',
 'This process is automatic. Your browser will redirect to your requested content shortly.',
 'This process is automatic. Your browser will redirect to your requested content shortly.',
 'This process is automatic. Your browser will redirect to your requested content shortly.',
 'This process is automatic. Your browser will redirect to your requested content shortly.',
 'This process is automatic. Your browser will redirect to your requested content shortly.',
 'This process is automatic. Your browser will redirect to your requested content shortly.',
 'This process is automatic. Your browser will redirect to your requested content shortly.',
 'This process is automatic. Your browser will redirect to your requested content shortly.',
 'This process is automatic. Your browser will redirect to your requested content shortly.',
 'This process is automatic. Your browser will redirect to your reques

In [43]:
realclear.iloc[100:110]

,doc_id,doc_source,doc_title,doc_content,doc_date,doc_url
100,1530,Real Clear Politics,Navy Chief Impatient to Avoid Being Out-Sticked,"Dustin Walker, RCDThe Navy’s top officer says ...",11/19/2013,http://www.realcleardefense.com/articles/2013/...
101,1531,Real Clear Politics,"My First Plane Flight, to Mourn JFK","Walter Shapiro, Yahoo! NewsA child’s first pla...",11/19/2013,http://www.realclearpolitics.com/2013/11/19/my...
102,1532,Real Clear Politics,A New GOP Excuse for Doing Nothing,NaN,11/19/2013,http://www.realclearpolitics.com/2013/11/19/a_...
103,1533,Real Clear Politics,"Suddenly, It's OK to Talk Obamacare Repeal",NaN,11/19/2013,http://www.realclearpolitics.com/2013/11/19/su...
104,1534,Real Clear Politics,Rein In Reckless Public Pensions,NaN,11/19/2013,http://www.realclearpolitics.com/2013/11/19/re...
105,1535,Real Clear Politics,Lincoln's Spare Words at Gettysburg Still Inspire,NaN,11/19/2013,http://www.realclearpolitics.com/2013/11/19/li...
106,1763,Real Clear Politics,Exposing the CIA's Stain on America,"Ruth Marcus, Washington PostWASHINGTON -- Rele...",12/10/2014,http://www.realclearpolitics.com/articles/2014...
107,1770,Real Clear Politics,Partisan Report Fails America,"Bob Kerrey, USA TodayI regret having to write ...",12/10/2014,http://www.realclearpolitics.com/2014/12/10/pa...
108,1784,Real Clear Politics,DC Politicians Threw the CIA Under the Bus,"Jose Rodriguez, FOX NewsYes. I mean, I think i...",12/10/2014,http://www.realclearpolitics.com/video/2014/12...
109,1811,Real Clear Politics,Here Come the Torture Apologists,"Andrew Rosenthal, New York TimesThe publicatio...",12/10/2014,http://www.realclearpolitics.com/2014/12/10/he...


In [45]:
articles[102:110]

['With unrestrained glee, Republicans are using the calamitous debut of the Affordable Care Act as their latest justification for undermining all of health care reform. But they’re not stopping there. The Obama administration’s fumbling is apparently a good excuse for them to do nothing on immigration reform, on a budget agreement, and on any other initiative coming out of the White House.',
 'Repeal: Not too long ago, Republicans pushing to kill ObamaCare were called extremists, obsessives and traitors. Now even liberals are discussing the possibility of repeal. We\'ve reached an important turning point. Back in May, nobody batted an eye when Bill Maher blasted the GOP\'s efforts to repeal ObamaCare. "At some point," he said, "obstruction becomes, I don\'t know, treason?"',
 "This month's races for governor sent a widely noted message that Republicans will do much better if they nominate center-right candidates like New Jersey's Chris Christie than far-right candidates like Virginia's

In [42]:
len(links)

100